In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math

In [2]:
import devanalyst.simulation.statics as S_
from devanalyst.simulation.simulationModels import Distribution
from devanalyst.simulation.generateTimecards import ReleaseLog

importing Jupyter notebook from c:\alejandro\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\statics.ipynb
importing Jupyter notebook from c:\alejandro\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\simulationModels.ipynb
importing Jupyter notebook from c:\alejandro\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\businessObjects.ipynb
importing Jupyter notebook from c:\alejandro\code\chateauclaudia-labs\devanalyst\devanalyst\simulation\generateTimecards.ipynb


<h1>Timecard Visualizations</h1>

<h2>Visualize runReleaseCycle's log</h2>

In [2]:
class VisualizeWork:
    
    def render(start_df, end_df, ax, target, include_legend):
        # RGB
        lime        = [0/256, 255/256, 0/256,1]
        light_green = [0/256, 220/256, 0/256,1]
        green       = [0/256, 128/256, 0/256,1]
        
        amber       = [255/256,191/256,0/256,1]
        dark_amber  = [255/256,100/256,0/256,1]
        red         = [255/256, 0/256, 0/256,1]
        dark_red    = [102/256, 3/256, 8/256,1]
        
        aqua        = [0/256,255/256,255/256, 1]
        blue        = [0/256,0/256,255/256, 1]
        purple      = [128/256, 0/256, 255/256, 1]
        
        light_blue  = [0/256,200/256,255/256, 1]
        
        # Log categories in backlog_df appear in this order:
        # prior_progressed, prior_remaining, new_work, regressed
        start_colors = np.array([blue, light_blue, aqua, dark_red])
        start_cmp = ListedColormap(start_colors)
        start_ax = start_df.set_index('Breakout').T.plot(kind='bar', stacked=True, ax=ax,  legend=include_legend,\
                                                           colormap = start_cmp, position = 1.0, width=0.35)
        start_ax.set(xlabel = 'Estimate', ylabel = '# of stories')
        
        # Log categories in backlog_df appear in this order:
        # completed, progressed, to_finish, not_started, over_budget, regressed
        end_colors = np.array([green, dark_amber, amber, purple, red, dark_red])
        end_cmp = ListedColormap(end_colors)
        end_ax = end_df.set_index('Breakout').T.plot(kind='bar', stacked=True, ax=start_ax,   legend=include_legend,\
                                                           colormap = end_cmp, position = 0.0, width=0.35)
        end_ax.set(xlabel = 'Estimate', ylabel = '# of stories')

        started, finished, mean, distance = VisualizeWork._calc_stats(start_df, end_df, target) 
        
        end_ax.set_xlabel('planned=' + str(round(started,2)) + '; mean=' + str(round(mean, 2)) + 
                          '; dist=' + str(round(distance,2)) + '; did=' + str(round(finished, 2)))

        max_val = math.ceil(end_df.drop(columns = ['Breakout']).sum().max())       
        end_ax.set_yticks(range(1,max_val+1), minor=True)
        end_ax.grid(b=True, which='minor', axis='y', linestyle=':', linewidth=0.75)
        
        if (include_legend):
            end_ax.legend(loc='center left', bbox_to_anchor=(0.0, 1.4), ncol=3)
                
    def render_backlog(backlog_df, ax, target, include_legend):
        # RGB
        lime        = [0/256, 255/256, 0/256,1]
        light_green = [0/256, 220/256, 0/256,1]
        green       = [0/256, 128/256, 0/256,1]
        
        amber       = [255/256,191/256,0/256,1]
        dark_amber  = [255/256,100/256,0/256,1]
        red         = [255/256, 0/256, 0/256,1]
        dark_red    = [102/256, 3/256, 8/256,1]
        gray        = [128/256, 128/256, 128/256,1]
        
        aqua        = [0/256,255/256,255/256, 1]
        blue        = [0/256,0/256,255/256, 1]
        purple      = [128/256, 0/256, 255/256, 1]
        
        light_blue = [0/256,200/256,255/256, 1]
        
        # WATCH OUT:
        # backlog colors should be listed in the order corresponding to the log category they correspond to.
        # Log categories in backlog_df appear in this order:
        # unplanned, completed, planned_progressed, planned_remaining, not_started, regressed
        backlog_colors = np.array([gray, green, dark_amber, amber, purple, dark_red])
        
        backlog_cmp = ListedColormap(backlog_colors)
        backlog_ax = backlog_df.set_index('Breakout').T.plot(kind='bar', stacked=True, ax=ax, legend=include_legend, \
                                                           colormap = backlog_cmp, position = 0.0, width=0.35)
        backlog_ax.set(xlabel = 'Estimate', ylabel = '# of stories')
        
        target_dist, total, mean = VisualizeWork._getTargetDist(backlog_df)    
        distance = Distribution.measureDistributionDistance(target_dist, target)
        backlog_ax.set_xlabel('total=' + str(round(total,2)) + '; mean=' + str(round(mean, 2)) + '; distance=' + str(round(distance,2)))

        max_val = math.ceil(backlog_df.drop(columns = ['Breakout']).sum().max())       
        backlog_ax.set_yticks(range(1,max_val+1), minor=True)
        backlog_ax.grid(b=True, which='minor', axis='y', linestyle=':', linewidth=0.75)

        if (include_legend):
            backlog_ax.legend(loc='center left', bbox_to_anchor=(0.0, 1.4), ncol=2)
    
    def _calc_stats(start_df, end_df, target):
        dist_df = start_df[start_df[ReleaseLog.BREAKOUT].isin([ReleaseLog.PRIOR_TO_FINISH, 
                                                               ReleaseLog.NEW_WORK,
                                                              ReleaseLog.REGRESSED])].sum().drop(ReleaseLog.BREAKOUT)
        dist, total_start, mean = VisualizeWork._df_to_dist(dist_df) #dist is a dictionary representing a distribution
                
        distance = Distribution.measureDistributionDistance(dist, target)

        dist_df = end_df[end_df[ReleaseLog.BREAKOUT].isin([ReleaseLog.COMPLETED, 
                                                           ReleaseLog.PROGRESSED])].sum().drop(ReleaseLog.BREAKOUT)
        x, acc_end, y = VisualizeWork._df_to_dist(dist_df) #dist is a dictionary representing a distribution

        dist_df = start_df[start_df[ReleaseLog.BREAKOUT].isin([ReleaseLog.PRIOR_PROGRESSED])].sum().drop(ReleaseLog.BREAKOUT)
        x, prior_end, y = VisualizeWork._df_to_dist(dist_df) #dist is a dictionary representing a distribution
        
        total_end = acc_end - prior_end

        return total_start, total_end, mean, distance

    def _getTargetDist(backlog_df):
        target_df = backlog_df[backlog_df[ReleaseLog.BREAKOUT].isin([ReleaseLog.UNPLANNED, 
                                                                     ReleaseLog.TO_FINISH, 
                                                                     ReleaseLog.NOT_STARTED,
                                                                    ReleaseLog.REGRESSED])].sum().drop(ReleaseLog.BREAKOUT)
        target, total_stories, mean = VisualizeWork._df_to_dist(target_df) #target is a dictionary representing a distribution

        return target, total_stories, mean 
        
    def _df_to_dist(dist_df):
        dist = {}
        total_stories = 0.0
        weighted_stories = 0.0
        for e in dist_df.index:
            assert(type(e)==int) #If e is not an int, then it is a spurious columns that should have been removed earlier
            #if type(e) != int:
            #    continue # Fixes a bug. This colums is not really an estimate, possibly an added label/description
            stories = dist_df[e]
            dist[e] = stories
            total_stories += stories
            weighted_stories += e * stories
        if total_stories != 0:
            mean = weighted_stories/total_stories
        else:
            mean = 0
        return dist, total_stories, mean
    
    def _remove_spurious_colums(df, spurious_cols):
        cols_to_drop = []
        for c in spurious_cols:
            if c in df.columns:
                cols_to_drop.append(c)
        df = df.drop(columns=cols_to_drop)
        return df
        

In [4]:
def renderReleaseCycleLog(teamId, release_log, first, last, spurious_columns=[]):
# -release_log: a ReleaseLog instance.
    
    team_log = release_log.log[teamId] # release_log is a ReleaseLog instance, and 'team_log' is a dict
    sprints = list(team_log.keys())
    
    for sprint in sprints:
        if sprint < first or sprint > last: # Only run for the chosen sprints
            continue
            
        if sprint==first:
            include_legend = True
        else:
            include_legend = False
            
        fig, axs = plt.subplots(1, 3)   

        fig.suptitle('----------------------------------------------------- SPRINT '+ str(sprint) + 
                     ' -----------------------------------------------------', fontsize=16, y=0.95, x=1)

        backlog_df                              = team_log[sprint]['backlog']
        backlog_df                              = VisualizeWork._remove_spurious_colums(backlog_df, spurious_columns)
        target, target_total, target_mean       = VisualizeWork._getTargetDist(backlog_df)
        VisualizeWork.render_backlog(backlog_df, axs[0], target, include_legend)

        start_df                                = team_log[sprint]['planned_Start_CURRENT_SPRINT']
        start_df                                = VisualizeWork._remove_spurious_colums(start_df, spurious_columns)
        end_df                                  = team_log[sprint]['planned_End_CURRENT_SPRINT']
        end_df                                  = VisualizeWork._remove_spurious_colums(end_df, spurious_columns)
        VisualizeWork.render(start_df, end_df, axs[1], target, include_legend)

        start_next_df                           = team_log[sprint]['planned_Start_NEXT_SPRINT']
        start_next_df                           = VisualizeWork._remove_spurious_colums(start_next_df, spurious_columns)
        end_next_df                             = team_log[sprint]['planned_End_NEXT_SPRINT']
        end_next_df                             = VisualizeWork._remove_spurious_colums(end_next_df, spurious_columns)
        VisualizeWork.render(start_next_df, end_next_df, axs[2], target, False)

        plt.subplots_adjust(top=0.8, right=2.3, wspace=0.3, hspace=1.5)